# Analyse de données et modèle de prédiction

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when

### Initialize Spark session

In [ ]:
spark = SparkSession.builder \
    .appName("Vehicle Category Prediction") \
    .getOrCreate()

In [ ]:
def print_df(df):
    print("Size", df.count())
    df.printSchema()
    df.show(5)

def load_data_in_csv_file(file_path):
    df = spark.read.options(delimiter=",", header=True, inferSchema=True).csv(file_path)

    # Display schema and first five rows of the DataFrame
    print("Schema and first rows in", file_path)
    print_df(df)
    
    return df

### Load the dataset

In [ ]:
# File path in Hadoop FS
hdfs_path = "hdfs:///tpa_groupe_14/data/fusion/fusion.csv"

# Load CSV data into a DataFrame
vehicles_df = load_data_in_csv_file(hdfs_path)

### Data formatting

In [ ]:
column_names = vehicles_df.schema.names

for column_name in column_names:
    vehicles_df = vehicles_df.withColumnRenamed(column_name, column_name.replace(".", "_"))

print_df(vehicles_df)

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StringIndexer

### Optional

In [ ]:
# Optional: Sub-sample the DataFrame
vehicles_df = vehicles_df.sample(withReplacement=False, fraction=0.3, seed=42)

In [ ]:
# Prepare features
indexers = [
    StringIndexer(inputCol=column, outputCol=column+"_index").fit(vehicles_df)
    for column in ["client_view_sexe", "client_view_situationfamiliale", "immatriculation_co2_view_categorie"]
]

assembler = VectorAssembler(
    inputCols=[
        "client_view_age", "client_view_taux", "client_view_nbenfantsacharge", "client_view_deuxiemevoiture",
        "client_view_sexe_index", "client_view_situationfamiliale_index"
    ],
    outputCol="features"
)

# Split the DataFrame into training and test sets (80% training, 20% test)
train_df, test_df = vehicles_df.randomSplit([0.8, 0.2], seed=42)

pipeline = Pipeline(stages=indexers + [assembler])
pipeline_model = pipeline.fit(train_df)

train_df = pipeline_model.transform(train_df)
test_df = pipeline_model.transform(test_df)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.mllib.evaluation import MulticlassMetrics

# La précision & Matrice de confusion

In [ ]:
# Evaluation function
def evaluate_classifier(classifier, param_grid, train_df, test_df, title):
    evaluator = MulticlassClassificationEvaluator(labelCol="immatriculation_co2_view_categorie_index", predictionCol="prediction", metricName="accuracy")
    
    # Create the cross-validator
    cross_validator = CrossValidator(
        estimator=classifier,
        estimatorParamMaps=param_grid,
        evaluator=evaluator,
        numFolds=3, 
        seed=42
    )
    
    # Train the model with the best hyperparameters
    cv_model = cross_validator.fit(train_df)
    
    # Make predictions on the test data
    predictions = cv_model.transform(test_df)
    
    # Evaluate the model
    accuracy = evaluator.evaluate(predictions)
    
    # Calcul de la matrice de confusion
    predictions_and_labels = predictions.select("prediction", "immatriculation_co2_view_categorie_index")
    prediction_rdd = predictions_and_labels.rdd.map(tuple)
    
    metrics = MulticlassMetrics(prediction_rdd)
    confusion_matrix = metrics.confusionMatrix().toArray()

    # Calcul des taux de succès par classe
    labels = metrics.labels
    per_class_metrics = {}
    for label in labels:
        per_class_metrics[label] = {
            "precision": metrics.precision(label),
            "recall": metrics.recall(label),
            "f1_score": metrics.fMeasure(label)
        }
    
    # Calcul des métriques globales pondérées
    weighted_metrics = {
        "precision": metrics.weightedPrecision,
        "recall": metrics.weightedRecall,
        "f1_score": metrics.weightedFMeasure(),
        "accuracy": accuracy
    }

    print(f"{title} Accuracy = {:.2f}".format(accuracy))
    print(f"{title} Confusion Matrix:\n", confusion_matrix)

        
    print("\nPar Classe Metrics:")
    for label, metrics in per_class_metrics.items():
        print(f" Class {label}:")
        print(f"  Precision: {metrics['precision']:.2f}")
        print(f"  Recall: {metrics['recall']:.2f}")
        print(f"  F1 Score: {metrics['f1_score']:.2f}")
    
    print("\nWeighted Metrics:")
    print(f"  Weighted Precision: {weighted_metrics['precision']:.2f}")
    print(f"  Weighted Recall: {weighted_metrics['recall']:.2f}")
    print(f"  Weighted F1 Score: {weighted_metrics['f1_score']:.2f}")
    print(f"  Accuracy: {weighted_metrics['accuracy']:.2f}")
    
    return cv_model

In [ ]:
from pyspark.ml.classification import (
    DecisionTreeClassifier, RandomForestClassifier, GBTClassifier, 
    LinearSVC, LogisticRegression, MultilayerPerceptronClassifier, NaiveBayes, OneVsRest
)

1. Random Forest

In [ ]:
rf = RandomForestClassifier(labelCol="immatriculation_co2_view_categorie_index", featuresCol="features")

# Define the hyperparameter grid
rf_param_grid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [10, 20]) \
    .addGrid(rf.maxDepth, [5, 10]) \
    .build()

rf_model = evaluate_classifier(
    classifier=rf,
    param_grid=rf_param_grid,
    train_df=train_df,
    test_df=test_df;
    title='RandomForestClassifier'
)


2. Decision Tree

In [ ]:
dt = DecisionTreeClassifier(labelCol="immatriculation_co2_view_categorie_index", featuresCol="features")

dt_param_grid = ParamGridBuilder()\
    .addGrid(dt.maxDepth, [5, 10])\
    .addGrid(dt.impurity, ["gini", "entropy"])\
    .build()

dt_model = evaluate_classifier(
    classifier=dt,
    param_grid=dt_param_grid,
    train_df=train_df,
    test_df=test_df,
    title='DecisionTreeClassifier'
)


3. Gradient Boosted Trees

In [ ]:
gbt = GBTClassifier(labelCol="immatriculation_co2_view_categorie_index", featuresCol="features")
ovr = OneVsRest(classifier=gbt, labelCol="immatriculation_co2_view_categorie_index", featuresCol="features")

gbt_param_grid = ParamGridBuilder()\
    .addGrid(gbt.maxIter, [10, 20])\
    .addGrid(gbt.maxDepth, [5, 10])\
    .build()

gbt_model = evaluate_classifier(
    classifier=ovr,
    param_grid=gbt_param_grid,
    train_df=train_df,
    test_df=test_df,
    title='GBTClassifier'
)


4. Support Vector Machine

In [ ]:
svm = LinearSVC(labelCol="immatriculation_co2_view_categorie_index", featuresCol="features")
ovr = OneVsRest(classifier=svm, labelCol="immatriculation_co2_view_categorie_index", featuresCol="features")

svm_param_grid = ParamGridBuilder()\
    .addGrid(svm.maxIter, [10, 20])\
    .addGrid(svm.regParam, [0.01, 0.1])\
    .build()

svm_model = evaluate_classifier(
    classifier=ovr,
    param_grid=svm_param_grid,
    train_df=train_df,
    test_df=test_df,
    title='LinearSVC'
)

5. Logistic Regression

In [ ]:
lr = LogisticRegression(labelCol="immatriculation_co2_view_categorie_index", featuresCol="features")

lr_param_grid = ParamGridBuilder()\
    .addGrid(lr.maxIter, [10, 20])\
    .addGrid(lr.regParam, [0.01, 0.1])\
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
    .build()

lr_model = evaluate_classifier(
    classifier=lr,
    param_grid=lr_param_grid,
    train_df=train_df,
    test_df=test_df,
    title='LogisticRegression'
)

6. Neural Networks (Multilayer Perceptron Classifier)

In [ ]:
num_classes = train_df.select("immatriculation_co2_view_categorie_index").distinct().count()

mlp = MultilayerPerceptronClassifier(labelCol="immatriculation_co2_view_categorie_index", featuresCol="features", layers=[6, 5, 4, num_classes])

mlp_param_grid = ParamGridBuilder()\
    .addGrid(mlp.maxIter, [50, 100])\
    .build()

mlp_model = evaluate_classifier(
    classifier=mlp,
    param_grid=mlp_param_grid,
    train_df=train_df,
    test_df=test_df,
    title='MultilayerPerceptronClassifier'
)


7. Naive Bayes

In [ ]:
nb = NaiveBayes(labelCol="immatriculation_co2_view_categorie_index", featuresCol="features")

nb_param_grid = ParamGridBuilder()\
    .addGrid(nb.smoothing, [0.5, 1.0, 1.5])\
    .build()

nb_model = evaluate_classifier(
    classifier=nb,
    param_grid=nb_param_grid,
    train_df=train_df,
    test_df=test_df,
    title='NaiveBayes'
)

# Prediction

- Charger Marketing

In [ ]:
from pyhive import hive

In [ ]:
hive_host = 'localhost'
hive_port = 10000
hive_username = ' ' 
hive_password = ' '

In [ ]:
#conn = None
try:
    # Établir une connexion avec authentification LDAP
    conn = hive.Connection(
        host=hive_host,
        port=hive_port,
        username=hive_username,
        password=hive_password,
        auth='LDAP'  
    )
    print("Connecté à Hive avec succès")
except Exception as e:
    print(f"Erreur lors de la connexion à Hive: {e}")

In [ ]:
# Créer un curseur
cursor=conn.cursor()

In [ ]:
def findAll(table) :
    # Exécuter une requête pour récupérer les données de la table "catalogue"
    query = "SELECT * FROM " + table
    cursor.execute(query)

    # Charger les résultats dans un DataFrame Pandas
    data = cursor.fetchall()

    # Récupérer les noms des colonnes
    columns = [desc[0] for desc in cursor.description] 
    df = pd.DataFrame(data, columns=columns)
    print("Select all ",table)

    display(df.head())
    
    return df

In [ ]:
marketing_df = findAll("marketing_view")

Prétraiter les Données Marketing

In [ ]:
# Renommer les colonnes pour éviter les conflits
for column_name in marketing_df.columns:
    marketing_df = marketing_df.withColumnRenamed(column_name, column_name.replace(".", "_"))

# Utiliser le pipeline formé précédemment pour transformer les données
marketing_df = pipeline_model.transform(marketing_df)


Appliquer le Modèle de Prédiction

In [ ]:
# Utiliser le modèle formé pour faire des prédictions sur les données Marketing
predictions = rf_model.transform(marketing_df)

# Sélectionner les colonnes pertinentes pour l'affichage
results = predictions.select("client_view_id", "prediction")


Exporter les Résultats

In [ ]:
# Chemin de sortie pour les résultats
results_hdfs_path = "hdfs:///tpa_groupe_14/results/marketing_predictions.csv"

# Sauvegarder les résultats dans HDFS
results.write.csv(results_hdfs_path, header=True)
